In [2]:
%pwd

'/Users/anwinsanju/Code/Projects/End-to-end-Mental-Health-Chatbot-Gen-AI/research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'/Users/anwinsanju/Code/Projects/End-to-end-Mental-Health-Chatbot-Gen-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [38]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [39]:
extracted_data=load_pdf_file(data='Data/')

In [12]:
# extracted_data

In [40]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [41]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [29]:
# text_chunks

In [42]:
from langchain.embeddings import HuggingFaceEmbeddings

In [44]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', )
    return embeddings


In [34]:
import sentence_transformers
print(sentence_transformers.__version__)

3.4.1


In [51]:
embeddings = download_hugging_face_embeddings()

In [52]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [54]:
# query_result

In [67]:
from dotenv import load_dotenv
load_dotenv()

True

In [68]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [57]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [69]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [59]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [61]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [62]:
docsearch

In [63]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [64]:
retrieved_docs = retriever.invoke("What is Acne?")

In [65]:
retrieved_docs

[Document(id='c148931d-5683-4ef6-9d11-96210f76d9ed', metadata={'page': 39.0, 'page_label': '40', 'source': 'Data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='50ac4e9b-599e-4192-b22c-638a71b090fe', metadata={'page': 38.0, 'page_label': '39', 'source': 'Data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='92909809-237f-4491-bd9d-e56252f328c2', metadata={'page': 37.0, 'page_label': '38', 'source': 'Data/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized

In [70]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [71]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [72]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [76]:
# response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
# print(response["answer"])

In [75]:
# response = rag_chain.invoke({"input": "What is stats?"})
# print(response["answer"])